<a href="https://colab.research.google.com/github/julipreciado/Proyecto-IA/blob/main/02_preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Análisis global de las columnas del dataset**

En esta sección se realiza un **análisis exploratorio general** de las variables del conjunto de datos con el fin de comprender su estructura y calidad.  
Se examinan aspectos como:

- **Tipos de datos:** Se identifican las columnas numéricas, categóricas y de texto, lo que permite definir estrategias posteriores de limpieza y codificación.
- **Distribución y completitud:** Se evalúa la presencia de valores nulos, la consistencia de los datos y la cantidad de registros válidos por variable.
- **Rangos y valores únicos:** Se revisan las posibles anomalías o columnas con información redundante o poco informativa.
- **Relevancia o técnica:** Se realiza una primera interpretación del papel de cada variable dentro del contexto del proyecto, diferenciando las de entrada (predictoras) de la variable objetivo.

Este análisis proporciona una **visión general del estado del dataset** y orienta las decisiones de preprocesamiento y modelado que se desarrollarán en las siguientes etapas.


In [ ]:
# === preprocesado - Notebook ===
# Proyecto: preprocesado

# === 0. Requisitos / Montar Google Drive ===
from google.colab import drive
drive.mount('/content/drive')

# === 1. Parámetros ===
TRAIN_PATH = '/content/drive/MyDrive/Introducción a inteligencia artificial/Proyecto/train.csv'
TEST_PATH  = '/content/drive/MyDrive/Introducción a inteligencia artificial/Proyecto/test.csv'
SAMPLE_SUB_PATH = '/content/drive/MyDrive/Introducción a inteligencia artificial/Proyecto/submission_example.csv'

# Columnas principales
TARGET_COL = 'RENDIMIENTO_GLOBAL'   # columna objetivo
SUB_ID_COL = 'ID'                   # columna identificadora
SUB_PRED_COL = None                 # se infiere si None

# === 2. Librerías y carga de archivos ===
import pandas as pd
import numpy as np
import os
from datetime import datetime
from IPython.display import display

def safe_read(path):
    """Lee CSV y maneja errores básicos de lectura."""
    try:
        return pd.read_csv(path)
    except Exception as e:
        print(f"Error al leer {path}: {e}")
        raise

print("Cargando archivos...")
train = safe_read(TRAIN_PATH)
test  = safe_read(TEST_PATH)
sample_sub = safe_read(SAMPLE_SUB_PATH)
print("Archivos cargados correctamente.")

# === 3. Resumen general de estructura ===
print("\n--- Información general ---")
print(f"Train shape: {train.shape}")
print(f"Test  shape: {test.shape}")
print(f"Sample submission shape: {sample_sub.shape}")

# === 4. Función de resumen por columnas ===
def summary_df(df):
    """Genera resumen por columna: tipo, nulos, únicos y ejemplos."""
    n = df.shape[0]
    resumen = []
    for c in df.columns:
        s = df[c]
        resumen.append({
            'columna': c,
            'tipo': str(s.dtype),
            'faltantes': int(s.isna().sum()),
            '%_faltantes': round(100 * s.isna().mean(), 2),
            'valores_unicos': int(s.nunique(dropna=True)),
            'muestras': ", ".join(map(str, s.dropna().unique()[:5]))
        })
    return pd.DataFrame(resumen)

# === 5. Resumen de train/test ===
print("\n--- Resumen del conjunto de entrenamiento ---")
train_summary = summary_df(train)
display(train_summary.head(10))

print("\n--- Resumen del conjunto de prueba ---")
test_summary = summary_df(test)
display(test_summary.head(10))

print("\n--- Resumen del archivo de submission ---")
sample_summary = summary_df(sample_sub)
display(sample_summary)

# === 6. Distribución de la variable objetivo ===
print(f"\nDistribución de la variable objetivo '{TARGET_COL}':")
display(train[TARGET_COL].value_counts(dropna=False).head(20))

print("\nPorcentajes (top 20):")
display((100 * train[TARGET_COL].value_counts(dropna=False) / len(train)).round(2).head(20))

# === 7. Valores faltantes y tipos ===
print("\nTop 10 columnas con mayor porcentaje de valores faltantes:")
display(train_summary.sort_values('%_faltantes', ascending=False).head(10)[['columna', 'tipo', '%_faltantes', 'valores_unicos']])

print("\nConteo de tipos de datos en train:")
display(train_summary['tipo'].value_counts())

# === 9. Vista previa de los datos ===
print("\n--- Primeras filas del train ---")
display(train.head(3))

print("\n--- Primeras filas del test ---")
display(test.head(3))

print("\n--- Primeras filas del sample_submission ---")
display(sample_sub.head(5))



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Cargando archivos...
Archivos cargados correctamente.

--- Información general ---
Train shape: (692500, 21)
Test  shape: (296786, 20)
Sample submission shape: (296786, 2)

--- Resumen del conjunto de entrenamiento ---


,columna,tipo,faltantes,%_faltantes,valores_unicos,muestras
0,ID,int64,0,0.00,692500,"904256, 645256, 308367, 470353, 989032"
1,PERIODO_ACADEMICO,int64,0,0.00,9,"20212, 20203, 20195, 20183, 20194"
2,E_PRGM_ACADEMICO,object,0,0.00,948,"ENFERMERIA, DERECHO, MERCADEO Y PUBLICIDAD, AD..."
3,E_PRGM_DEPARTAMENTO,object,0,0.00,31,"BOGOTÁ, ATLANTICO, SANTANDER, ANTIOQUIA, HUILA"
4,E_VALORMATRICULAUNIVERSIDAD,object,6287,0.91,8,"Entre 5.5 millones y menos de 7 millones, Entr..."
5,E_HORASSEMANATRABAJA,object,30857,4.46,5,"Menos de 10 horas, 0, Más de 30 horas, Entre 2..."
6,F_ESTRATOVIVIENDA,object,32137,4.64,7,"Estrato 3, Estrato 4, Estrato 5, Estrato 2, Es..."
7,F_TIENEINTERNET,object,26629,3.85,2,"Si, No"
8,F_EDUCACIONPADRE,object,23178,3.35,12,"Técnica o tecnológica incompleta, Técnica o te..."
9,F_TIENELAVADORA,object,39773,5.74,2,"Si, No"



--- Resumen del conjunto de prueba ---


,columna,tipo,faltantes,%_faltantes,valores_unicos,muestras
0,ID,int64,0,0.00,296786,"550236, 98545, 499179, 782980, 785185"
1,PERIODO_ACADEMICO,int64,0,0.00,9,"20183, 20203, 20212, 20195, 20196"
2,E_PRGM_ACADEMICO,object,0,0.00,919,"TRABAJO SOCIAL, ADMINISTRACION COMERCIAL Y DE ..."
3,E_PRGM_DEPARTAMENTO,object,0,0.00,31,"BOLIVAR, ANTIOQUIA, BOGOTÁ, SUCRE, ATLANTICO"
4,E_VALORMATRICULAUNIVERSIDAD,object,2723,0.92,8,"Menos de 500 mil, Entre 2.5 millones y menos d..."
5,E_HORASSEMANATRABAJA,object,13379,4.51,5,"Menos de 10 horas, Entre 21 y 30 horas, 0, Ent..."
6,F_ESTRATOVIVIENDA,object,13795,4.65,7,"Estrato 3, Estrato 2, Estrato 1, Estrato 4, Es..."
7,F_TIENEINTERNET,object,11539,3.89,2,"Si, No"
8,F_EDUCACIONPADRE,object,9993,3.37,12,"Técnica o tecnológica completa, Secundaria (Ba..."
9,F_TIENELAVADORA,object,17259,5.82,2,"Si, No"



--- Resumen del archivo de submission ---


...

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264



--- Primeras filas del test ---


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,Si,Si,Si,Primaria completa,0.328,0.219,0.317,0.247
1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Técnica o tecnológica completa,0.227,0.283,0.296,0.324
2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.285,0.228,0.294,0.247



--- Primeras filas del sample_submission ---


,ID,RENDIMIENTO_GLOBAL
0,550236,medio-bajo
1,98545,medio-bajo
2,499179,medio-alto
3,782980,alto
4,785185,medio-bajo



# **Análisis de tipos de variables y cardinalidad categórica**

En este bloque se realiza una **clasificación general de las columnas** del conjunto de datos en numéricas y categóricas, excluyendo tanto el identificador (`ID`) como la variable objetivo (`RENDIMIENTO_GLOBAL`).  
El propósito es **comprender la estructura de los predictores** antes del preprocesamiento y determinar cuáles son útiles para el entrenamiento del modelo.

---

### **Procedimiento realizado**
1. Se separaron las variables predictoras (`X`) de la variable objetivo (`y`).
2. Se identificaron las columnas **numéricas** (`int64`, `float64`) y **categóricas** (`object`).
3. Se calculó la **cardinalidad** de cada variable categórica, es decir, la cantidad de valores únicos no nulos por columna.
4. Se organizó la información en un DataFrame resumen para visualizar fácilmente las variables con mayor y menor diversidad de valores.

---

### **Resultados principales**
- Se identificaron **6 variables numéricas** y un conjunto amplio de variables categóricas.
- Algunas variables presentan **alta cardinalidad**, como `E_PRGM_ACADEMICO` (948 valores únicos), mientras que otras tienen **baja cardinalidad** (≤2 valores únicos), por ejemplo:
  - `F_TIENEINTERNET`, `F_TIENEAUTOMOVIL`, `F_TIENELAVADORA`, `E_PRIVADO_LIBERTAD`, `E_PAGOMATRICULAPROPIO`, `F_TIENECOMPUTADOR`, `F_TIENEINTERNET.1`.

---

### **Conclusión**
A partir del análisis se decidió **no incluir en el entrenamiento** las variables categóricas con **baja cardinalidad** (dos o menos categorías), ya que aportan **escasa información discriminante** al modelo y pueden introducir **redundancia** o **ruido estadístico** en el proceso de aprendizaje.


In [ ]:
# identifico columnas numéricas y categóricas (excluyo ID y target)
target_col = 'RENDIMIENTO_GLOBAL'
id_col = 'ID'

X = train.drop(columns=[target_col])
y = train[target_col]

num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object']).columns.tolist()

# muestro resumen rápido
print("Numéricas:", num_cols)
print("Categoricas (ejemplo primeras 20):", cat_cols[:20])

# cardinalidades de categóricas
cardinality = {c: X[c].nunique(dropna=True) for c in cat_cols}
card_df = pd.DataFrame.from_dict(cardinality, orient='index', columns=['n_unique']).sort_values('n_unique', ascending=False)
display(card_df)

Numéricas: ['ID', 'PERIODO_ACADEMICO', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']
Categoricas (ejemplo primeras 20): ['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1', 'F_EDUCACIONMADRE']


,n_unique
E_PRGM_ACADEMICO,948
E_PRGM_DEPARTAMENTO,31
F_EDUCACIONMADRE,12
F_EDUCACIONPADRE,12
E_VALORMATRICULAUNIVERSIDAD,8
F_ESTRATOVIVIENDA,7
E_HORASSEMANATRABAJA,5
F_TIENEINTERNET,2
F_TIENEAUTOMOVIL,2
F_TIENELAVADORA,2



# **Imputación de valores faltantes en variables numéricas y categóricas**

# **Imputación de valores faltantes**

En esta sección se corrigen los valores nulos presentes en las variables predictoras (`X`) para asegurar un dataset completo y consistente antes del modelado.

---

### **Procedimiento**
- Se emplearon imputadores de `scikit-learn`:
  - **Numéricas:** reemplazo por la **media**.
  - **Categóricas:** reemplazo por el **valor más frecuente**.
- Se trabajó sobre una copia del dataset (`X_imputed`) y se verificó que todas las columnas quedaran sin valores nulos.

---

### **Resultado**
La imputación eliminó correctamente todos los valores faltantes, dejando el conjunto de datos **sin nulos** y listo para las siguientes etapas de preprocesamiento.


In [ ]:
from sklearn.impute import SimpleImputer

# imputadores
num_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='most_frequent')

# aplico a una copia para mantener original
X_imputed = X.copy()

# imputo numéricas
if len(num_cols) > 0:
    X_imputed[num_cols] = num_imputer.fit_transform(X_imputed[num_cols])

# imputo categóricas
if len(cat_cols) > 0:
    X_imputed[cat_cols] = cat_imputer.fit_transform(X_imputed[cat_cols])

# confirmo que ya no hay nulos en X_imputed
print("Nulos por columna después de imputar (debe ser 0):")
print(X_imputed.isna().sum().sort_values(ascending=False).head(10))

Nulos por columna después de imputar (debe ser 0):
ID                             0
PERIODO_ACADEMICO              0
E_PRGM_ACADEMICO               0
E_PRGM_DEPARTAMENTO            0
E_VALORMATRICULAUNIVERSIDAD    0
E_HORASSEMANATRABAJA           0
F_ESTRATOVIVIENDA              0
F_TIENEINTERNET                0
F_EDUCACIONPADRE               0
F_TIENELAVADORA                0
dtype: int64



# **Clasificación de variables categóricas según su cardinalidad**

En esta parte se separan las variables categóricas en dos grupos con base en su **número de valores únicos (cardinalidad)**, lo que orienta las estrategias de codificación que se aplicarán más adelante.

---

### **Procedimiento**
- Se definió un **umbral de cardinalidad** igual a 2.  
- Las columnas con **≤ 2 valores únicos** se clasificaron como de **baja cardinalidad**, y las que superan este umbral, como de **alta cardinalidad**.  
- Se listaron las primeras columnas de alta cardinalidad para revisión.


In [ ]:
# Defino umbral de cardinalidad para decidir estrategia
HIGH_CARD_THRESHOLD = 2

low_card_cols = [c for c in cat_cols if X_imputed[c].nunique() <= HIGH_CARD_THRESHOLD]
high_card_cols = [c for c in cat_cols if X_imputed[c].nunique() > HIGH_CARD_THRESHOLD]

print("Categoricas baja cardinalidad:", len(low_card_cols))
print("Categoricas alta cardinalidad:", len(high_card_cols))
print("Columnas alta cardinalidad (ejemplo):", high_card_cols[:10])

Categoricas baja cardinalidad: 7
Categoricas alta cardinalidad: 7
Columnas alta cardinalidad (ejemplo): ['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_EDUCACIONPADRE', 'F_EDUCACIONMADRE']



# **Codificación de variables categóricas de alta cardinalidad**

En esta etapa se aplica una **codificación por frecuencia (frequency encoding)** a las variables categóricas con alta cardinalidad, con el objetivo de transformarlas en valores numéricos sin perder información relevante sobre su distribución.

---

### **Procedimiento**
- Se creó una copia del dataset imputado (`X_enc`).
- Para cada columna de alta cardinalidad, se calculó la **frecuencia relativa** de cada categoría en el conjunto de entrenamiento.
- Los valores categóricos se reemplazaron por su frecuencia correspondiente.

---

### **Resultado**
Las variables categóricas de alta cardinalidad quedaron representadas por **valores numéricos continuos** proporcionales a su frecuencia, manteniendo la información estadística.



In [ ]:
# frequency encoding: reemplazo cada valor por su frecuencia relativa en el train
X_enc = X_imputed.copy()

for c in high_card_cols:
    freq = X_enc[c].value_counts(normalize=True)
    X_enc[c] = X_enc[c].map(freq).astype(float)

# confirmo
display(X_enc[high_card_cols].head(3))

,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_EDUCACIONPADRE,F_EDUCACIONMADRE
0,0.016801,0.407450,0.055581,0.125908,0.304238,0.032566,0.066781
1,0.076887,0.059235,0.184014,0.168303,0.304238,0.090968,0.039759
2,0.002465,0.407450,0.184014,0.404634,0.304238,0.218725,0.238856


# **Codificación One-Hot para variables de baja cardinalidad**

En este paso se realiza la **codificación one-hot** de las variables categóricas con baja cardinalidad, generando nuevas columnas binarias que representan cada categoría posible.

---

### **Procedimiento**
- Se utilizó `OneHotEncoder` de *scikit-learn* con los parámetros:
  - `handle_unknown='ignore'` para evitar errores ante categorías nuevas.
  - `sparse_output=False` para obtener una matriz densa.
- El codificador se ajustó únicamente sobre las columnas de **baja cardinalidad** (`low_card_cols`).
- Se generó un nuevo DataFrame (`ohe_df`) con las variables transformadas y sus nombres derivados.

---

### **Resultado**
La codificación produjo una representación **numérica y expandida** de las variables categóricas de baja cardinalidad, apta para combinarse con las demás variables del modelo sin pérdida de información.


In [ ]:
from sklearn.preprocessing import OneHotEncoder

# OneHotEncoder con handle_unknown='ignore' y sparse_output=False para obtener ndarray
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

if len(low_card_cols) > 0:
    # ajusto el encoder solo sobre train para las columnas low_card_cols
    ohe.fit(X_enc[low_card_cols])
    ohe_cols = ohe.get_feature_names_out(low_card_cols).tolist()
    ohe_array = ohe.transform(X_enc[low_card_cols])
    ohe_df = pd.DataFrame(ohe_array, columns=ohe_cols, index=X_enc.index)
else:
    ohe_df = pd.DataFrame(index=X_enc.index)
    ohe_cols = []

print("Dimensión repr. one-hot:", ohe_df.shape)

Dimensión repr. one-hot: (692500, 14)



# **Construcción del conjunto final de variables (`X_final`)**

En esta etapa se integran todas las variables procesadas para conformar el **dataset final de entrenamiento**, combinando la información numérica y categórica codificada.

---

### **Procedimiento**
- Se agruparon en una sola estructura:
  - Las **variables numéricas** originales.  
  - Las **categóricas de alta cardinalidad** transformadas por frecuencia.  
  - Las **categóricas de baja cardinalidad** codificadas mediante one-hot.
- Se concatenaron todas las partes por columnas, generando el DataFrame final `X_final`.

---

### **Resultado**
El conjunto `X_final` contiene **todas las variables relevantes en formato numérico**, listo para ser utilizado en el entrenamiento del modelo de predicción.


In [ ]:
# numéricas ya están en X_enc[num_cols]
final_parts = []

if len(num_cols) > 0:
    final_parts.append(X_enc[num_cols].reset_index(drop=True))

if len(high_card_cols) > 0:
    final_parts.append(X_enc[high_card_cols].reset_index(drop=True))

if not ohe_df.empty:
    final_parts.append(ohe_df.reset_index(drop=True))

X_final = pd.concat(final_parts, axis=1)
print("Dimensión de X_final:", X_final.shape)


Dimensión de X_final: (692500, 27)



# **Escalamiento de variables numéricas**

En esta fase se aplica un **escalamiento estandarizado** a las variables numéricas del conjunto `X_final` para mejorar la estabilidad y el rendimiento del modelo.

---

### **Procedimiento**
- Se utilizó `StandardScaler` de *scikit-learn* para transformar las variables numéricas.  
- Cada columna fue ajustada para tener **media 0** y **desviación estándar 1**, manteniendo su distribución relativa.  
- El escalamiento se aplicó únicamente sobre las variables numéricas identificadas previamente.

---

### **Resultado**
Las variables numéricas quedaron **normalizadas en escala comparable**, facilitando el entrenamiento de modelos sensibles a la magnitud de los datos.


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

if len(num_cols) > 0:
    X_final[num_cols] = scaler.fit_transform(X_final[num_cols])

print("Se escalaron las columnas numéricas. Ejemplo primeras filas:")
display(X_final[num_cols].head(3))


Se escalaron las columnas numéricas. Ejemplo primeras filas:


,ID,PERIODO_ACADEMICO,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,1.434424,1.294094,0.437002,-0.556223,0.813978,0.060296
1,0.527513,1.294094,0.346934,-0.481341,0.508180,0.016142
2,-0.652132,0.439801,0.232301,-0.492038,0.729034,0.016142


# **Codificación de la variable objetivo**

En este paso se transforma la variable de salida (`y`) desde etiquetas categóricas hacia valores numéricos enteros, permitiendo su uso directo por los algoritmos de aprendizaje automático.

---

### **Procedimiento**
- Se utilizó `LabelEncoder` de *scikit-learn* para asignar a cada clase un **entero único**.  
- Se generó un diccionario de correspondencia entre las **etiquetas originales** y sus **valores codificados**.

---

### **Resultado**
La variable objetivo quedó representada en formato **numérico discreto**, manteniendo el significado de cada clase y quedando lista para el entrenamiento supervisado del modelo.

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_enc = le.fit_transform(y)

mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Mapping etiqueta -> entero:", mapping)


Mapping etiqueta -> entero: {'alto': np.int64(0), 'bajo': np.int64(1), 'medio-alto': np.int64(2), 'medio-bajo': np.int64(3)}


# **División del conjunto de datos y guardado de preprocesamiento**

En esta etapa final del preprocesamiento se preparan los conjuntos de datos para entrenamiento y validación, y se guardan los objetos necesarios para reproducir el flujo en producción.

---

### **Procedimiento**
- Se dividió el conjunto en **entrenamiento (80%)** y **validación (20%)** utilizando `train_test_split` con estratificación por la variable objetivo.  
- Los datos se transformaron a formato `ndarray` para su uso con modelos de *Keras*.  
- Se guardaron los conjuntos resultantes (`X_train`, `X_val`, `y_train`, `y_val`) en archivos `.npy`.  
- También se almacenaron mediante `joblib` los objetos de preprocesamiento: imputadores, codificadores, escalador y columnas utilizadas, junto con el `LabelEncoder` de la variable objetivo.


In [ ]:
from sklearn.model_selection import train_test_split
import joblib

X_array = X_final.values  # ndarray para keras
y_array = y_enc

X_train, X_val, y_train, y_val = train_test_split(
    X_array, y_array, test_size=0.2, random_state=42, stratify=y_array
)

OUT_FOLDER = '/content/drive/MyDrive/Introducción a inteligencia artificial/Proyecto/processed'
os.makedirs(OUT_FOLDER, exist_ok=True)

np.save(os.path.join(OUT_FOLDER, 'X_train.npy'), X_train)
np.save(os.path.join(OUT_FOLDER, 'X_val.npy'), X_val)
np.save(os.path.join(OUT_FOLDER, 'y_train.npy'), y_train)
np.save(os.path.join(OUT_FOLDER, 'y_val.npy'), y_val)

# guardo objetos necesarios para reproducir preprocesamiento en producción
joblib.dump({'num_imputer': num_imputer, 'cat_imputer': cat_imputer,
             'ohe': ohe, 'high_cardinality_cols': high_card_cols,
             'low_card_cols': low_card_cols, 'num_cols': num_cols,
             'scaler': scaler}, os.path.join(OUT_FOLDER, 'preprocessing_objects.pkl'))

joblib.dump(le, os.path.join(OUT_FOLDER, 'label_encoder.pkl'))

print("Guardado en:", OUT_FOLDER)
print("Shapes guardados: X_train", X_train.shape, "X_val", X_val.shape, "X_test (pendiente crear) ...")


Guardado en: /content/drive/MyDrive/Introducción a inteligencia artificial/Proyecto/processed
Shapes guardados: X_train (554000, 27) X_val (138500, 27) X_test (pendiente crear) ...



# **Preprocesamiento del conjunto de prueba**

Se replicaron los pasos aplicados al conjunto de entrenamiento para asegurar coherencia en la fase de inferencia.

---

### **Procedimiento**
- Imputación de valores faltantes usando los mismos imputadores.  
- Codificación de **alta cardinalidad** mediante frecuencias del conjunto de entrenamiento.  
- Codificación **one-hot** para las variables de baja cardinalidad con el encoder previamente ajustado.  
- Escalamiento de las variables numéricas con el `StandardScaler` entrenado.  
- Combinación de todas las partes en `X_test_final` y guardado en formato `.npy`.

---

### **Resultado**
El conjunto de prueba quedó **totalmente preprocesado y numérico**, alineado con el formato de entrenamiento y listo para ser utilizado en la etapa de evaluación del modelo.


In [ ]:
# Reproduzco los mismos pasos para test: imputación, high-card freq-encoding, one-hot usando el encoder ajustado, scaling
test_df = test.copy()

# imputación
test_df[num_cols] = num_imputer.transform(test_df[num_cols])
test_df[cat_cols] = cat_imputer.transform(test_df[cat_cols])

# freq encoding para columnas de alta cardinalidad
for c in high_card_cols:
    # si aparece valor no visto, map dará NaN; lo sustituyo por 0 (frecuencia 0)
    freq = X_imputed[c].value_counts(normalize=True)
    test_df[c] = test_df[c].map(freq).fillna(0.0).astype(float)

# one-hot para low_card_cols usando encoder ajustado
if len(low_card_cols) > 0:
    test_ohe_array = ohe.transform(test_df[low_card_cols])
    test_ohe_df = pd.DataFrame(test_ohe_array, columns=ohe.get_feature_names_out(low_card_cols), index=test_df.index)
else:
    test_ohe_df = pd.DataFrame(index=test_df.index)

# construir test final
parts = []
if len(num_cols) > 0:
    parts.append(test_df[num_cols].reset_index(drop=True))
if len(high_card_cols) > 0:
    parts.append(test_df[high_card_cols].reset_index(drop=True))
if not test_ohe_df.empty:
    parts.append(test_ohe_df.reset_index(drop=True))

X_test_final = pd.concat(parts, axis=1)

# escalar numéricas con scaler entrenado
if len(num_cols) > 0:
    X_test_final[num_cols] = scaler.transform(X_test_final[num_cols])

# guardar
np.save(os.path.join(OUT_FOLDER, 'X_test.npy'), X_test_final.values)
print("X_test shape:", X_test_final.shape)
print("Guardado X_test.npy en:", OUT_FOLDER)


X_test shape: (296786, 27)
Guardado X_test.npy en: /content/drive/MyDrive/Introducción a inteligencia artificial/Proyecto/processed




---



---



---



# **Carga de los conjuntos de datos procesados**

En esta sección se cargan los archivos preprocesados almacenados previamente, necesarios para iniciar la etapa de entrenamiento y evaluación del modelo.

---

### **Procedimiento**
- Se definió la ruta del proyecto (`PROJ`) y se aseguraron las carpetas necesarias.  
- Se cargaron desde disco los conjuntos `X_train`, `X_val`, `y_train`, `y_val` y `X_test` en formato `.npy`.  
- Se imprimieron las dimensiones de cada conjunto para confirmar su correcta lectura.






In [ ]:
import numpy as np
import pandas as pd
import os
from pathlib import Path

PROJ = '/content/drive/MyDrive/Introducción a inteligencia artificial/Proyecto/processed'
os.makedirs(PROJ, exist_ok=True)

X_train = np.load(os.path.join(PROJ, 'X_train.npy'))
X_val   = np.load(os.path.join(PROJ, 'X_val.npy'))
y_train = np.load(os.path.join(PROJ, 'y_train.npy'))
y_val   = np.load(os.path.join(PROJ, 'y_val.npy'))
X_test  = np.load(os.path.join(PROJ, 'X_test.npy'))

print("Shapes:")
print("X_train", X_train.shape)
print("X_val  ", X_val.shape)
print("y_train", y_train.shape)
print("X_test ", X_test.shape)


Shapes:
X_train (554000, 27)
X_val   (138500, 27)
y_train (554000,)
X_test  (296786, 27)


# **Carga del codificador de etiquetas y archivo de ejemplo de envío**

En esta sección se cargan los elementos necesarios para preparar la fase final de predicción y generación del archivo de envío.

---

### **Procedimiento**
- Se definió la ruta del archivo `label_encoder.pkl`, que contiene el objeto `LabelEncoder` entrenado durante el preprocesamiento.  
- Se cargó este codificador con `joblib.load()` para poder decodificar las etiquetas numéricas del modelo a sus nombres originales.  
- Se especificó la ubicación del archivo `submission_example.csv`, utilizado como plantilla para construir el archivo final de predicciones.  
- Finalmente, se visualizó la cabecera del archivo de ejemplo mediante `display(sample_sub.head())` para confirmar su estructura.



In [ ]:
import joblib

LE_PATH = '/content/drive/MyDrive/Introducción a inteligencia artificial/Proyecto/processed/label_encoder.pkl'
le = joblib.load(LE_PATH)

# sample submission path (ajusta si tu archivo está en otra ubicación)
SAMPLE_SUB_PATH = '/content/drive/MyDrive/Introducción a inteligencia artificial/Proyecto/submission_example.csv'
sample_sub = pd.read_csv(SAMPLE_SUB_PATH)
print("Sample submission head:")
display(sample_sub.head())


Sample submission head:


,ID,RENDIMIENTO_GLOBAL
0,550236,medio-bajo
1,98545,medio-bajo
2,499179,medio-alto
3,782980,alto
4,785185,medio-bajo


# **Codificación one-hot y cálculo de pesos de clase**

En esta sección se preparan las etiquetas para el entrenamiento del modelo, transformándolas a una representación adecuada y ajustando los pesos de clase para manejar posibles desbalances.

---

### **Procedimiento**
- Se determinó el número total de clases a partir del codificador de etiquetas (`len(le.classes_)`).  
- Se convirtieron las etiquetas `y_train` y `y_val` a formato **one-hot** utilizando la función `to_categorical()` de Keras, necesaria para modelos de clasificación multiclase.  
- Se calcularon los **pesos de clase** mediante `compute_class_weight('balanced', ...)`, con el fin de compensar posibles desequilibrios entre clases durante el entrenamiento.  
- Finalmente, los pesos se almacenaron en un diccionario (`class_weights`) para ser usados posteriormente por el modelo.


In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight

num_classes = len(le.classes_)
y_train_ohe = to_categorical(y_train, num_classes=num_classes)
y_val_ohe   = to_categorical(y_val, num_classes=num_classes)

# calcular class weights (por si las clases no estuvieran perfectamente balanceadas)
class_weights = compute_class_weight('balanced', classes=np.arange(num_classes), y=y_train)
class_weights = dict(enumerate(class_weights))
print("Num classes:", num_classes)
print("Class weights:", class_weights)


Num classes: 4
Class weights: {0: np.float64(0.9858002064130396), 1: np.float64(1.0007948551195895), 2: np.float64(1.0087767216577443), 3: np.float64(1.0049339718473371)}


# **Definición y compilación del modelo neuronal**

En esta sección se construye y compila la red neuronal que se utilizará para la clasificación multiclase de los pacientes según su GRD (Grupo Relacionado por el Diagnóstico).

---

### **Procedimiento**
- Se estableció una **semilla aleatoria** (`tf.random.set_seed(42)`) para garantizar la reproducibilidad de los resultados.  
- Se definió la función `build_model()` que construye un modelo **feed-forward** con las siguientes capas:
  - **Capa de entrada:** dimensión igual al número de características (`input_dim`).  
  - **Tres capas densas ocultas** con 256, 128 y 64 neuronas respectivamente, todas con activación *ReLU*.  
  - **Batch Normalization** y **Dropout** después de cada capa para mejorar la estabilidad y reducir el sobreajuste.  
  - **Capa de salida:** utiliza activación *softmax* para producir probabilidades sobre las `num_classes`.  
- Se compiló el modelo con:
  - Optimizador **Adam** (`learning_rate=1e-3`).  
  - Función de pérdida **categorical_crossentropy**.  
  - Métrica de evaluación **accuracy**.

---

### **Resultado**
El modelo quedó correctamente **definido y compilado**, mostrando una arquitectura **profunda y regularizada**, lista para ser entrenada sobre los conjuntos de datos preprocesados.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers

tf.random.set_seed(42)

input_dim = X_train.shape[1]

def build_model(input_dim, num_classes):
    inp = layers.Input(shape=(input_dim,))
    x = layers.Dense(256, activation='relu')(inp)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.25)(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    out = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs=inp, outputs=out)
    return model

model = build_model(input_dim, num_classes)
model.compile(optimizer=optimizers.Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 27)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │         7,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,372 (196.77 KB)

 Trainable params: 49,476 (193.27 KB)

 Non-trainable params: 896 (3.50 KB)


# **Entrenamiento del modelo neuronal**

En esta sección se entrena la red neuronal previamente definida, aplicando estrategias de regularización y control para optimizar su rendimiento y evitar el sobreajuste.

---

### **Procedimiento**
- Se definió la carpeta de salida `OUT_MODEL` para almacenar los modelos entrenados.  
- Se configuraron tres **callbacks** esenciales:
  - **ModelCheckpoint:** guarda automáticamente el modelo con la mejor *val_accuracy*.  
  - **EarlyStopping:** detiene el entrenamiento si no hay mejora en la precisión de validación durante 6 épocas consecutivas.  
  - **ReduceLROnPlateau:** reduce la tasa de aprendizaje a la mitad si la pérdida de validación no mejora en 3 épocas.  
- Se establecieron los parámetros de entrenamiento:
  - `BATCH_SIZE = 1024` y `EPOCHS = 50`.  
  - Se usó **class_weight** para compensar posibles desbalances entre clases.  
- Se entrenó el modelo con los conjuntos `X_train` y `X_val`, registrando el historial de métricas.

---

### **Resultado**
El modelo fue **entrenado exitosamente**, guardando automáticamente:
- El **mejor modelo** (`best_model.h5`) según la precisión de validación.  
- El **modelo final** (`final_model.h5`) al finalizar el proceso.  

La red quedó lista para su evaluación y análisis de desempeño.


In [ ]:
OUT_MODEL = '/content/drive/MyDrive/Introducción a inteligencia artificial/Proyecto/processed/nn_model'
os.makedirs(OUT_MODEL, exist_ok=True)

checkpoint = callbacks.ModelCheckpoint(
    filepath=os.path.join(OUT_MODEL, 'best_model.h5'),
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)
earlystop = callbacks.EarlyStopping(monitor='val_accuracy', patience=6, restore_best_weights=True, verbose=1)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6)

# Ajusta batch_size si la GPU/ram lo requiere. Con 554k ejemplos, 512-2048 son razonables.
BATCH_SIZE = 1024
EPOCHS = 50

history = model.fit(
    X_train, y_train_ohe,
    validation_data=(X_val, y_val_ohe),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[checkpoint, earlystop, reduce_lr],
    class_weight=class_weights,
    verbose=2
)

# guardar modelo final (el mejor ya se guardó como best_model.h5)
model.save(os.path.join(OUT_MODEL, 'final_model.h5'))


Epoch 1/50

Epoch 1: val_accuracy improved from -inf to 0.38794, saving model to /content/drive/MyDrive/Introducción a inteligencia artificial/Proyecto/processed/nn_model/best_model.h5


542/542 - 20s - 37ms/step - accuracy: 0.3889 - loss: 1.2796 - val_accuracy: 0.3879 - val_loss: 1.2788 - learning_rate: 5.0000e-04
Epoch 2/50

Epoch 2: val_accuracy improved from 0.38794 to 0.38856, saving model to /content/drive/MyDrive/Introducción a inteligencia artificial/Proyecto/processed/nn_model/best_model.h5


542/542 - 16s - 29ms/step - accuracy: 0.3895 - loss: 1.2794 - val_accuracy: 0.3886 - val_loss: 1.2784 - learning_rate: 5.0000e-04
Epoch 3/50

Epoch 3: val_accuracy improved from 0.38856 to 0.38866, saving model to /content/drive/MyDrive/Introducción a inteligencia artificial/Proyecto/processed/nn_model/best_model.h5


542/542 - 16s - 30ms/step - accuracy: 0.3891 - loss: 1.2792 - val_accuracy: 0.3887 - val_loss: 1.2784 - learning_rate: 5.0000e-04
Epoch 4/50

Epoch 4: val_accuracy did not improve from 0.38866
542/542 - 15s - 27ms/step - accuracy: 0.3891 - loss: 1.2791 - val_accuracy: 0.3875 - val_loss: 1.2785 - learning_rate: 5.0000e-04
Epoch 5/50

Epoch 5: val_accuracy did not improve from 0.38866
542/542 - 15s - 27ms/step - accuracy: 0.3893 - loss: 1.2788 - val_accuracy: 0.3883 - val_loss: 1.2780 - learning_rate: 5.0000e-04
Epoch 6/50

Epoch 6: val_accuracy did not improve from 0.38866
542/542 - 15s - 27ms/step - accuracy: 0.3890 - loss: 1.2787 - val_accuracy: 0.3881 - val_loss: 1.2776 - learning_rate: 5.0000e-04
Epoch 7/50

Epoch 7: val_accuracy improved from 0.38866 to 0.38874, saving model to /content/drive/MyDrive/Introducción a inteligencia artificial/Proyecto/processed/nn_model/best_model.h5


542/542 - 15s - 28ms/step - accuracy: 0.3898 - loss: 1.2783 - val_accuracy: 0.3887 - val_loss: 1.2779 - learning_rate: 5.0000e-04
Epoch 8/50

Epoch 8: val_accuracy improved from 0.38874 to 0.38934, saving model to /content/drive/MyDrive/Introducción a inteligencia artificial/Proyecto/processed/nn_model/best_model.h5


...

542/542 - 15s - 28ms/step - accuracy: 0.3920 - loss: 1.2748 - val_accuracy: 0.3919 - val_loss: 1.2736 - learning_rate: 2.5000e-04
Epoch 28/50

Epoch 28: val_accuracy did not improve from 0.39188
542/542 - 15s - 27ms/step - accuracy: 0.3926 - loss: 1.2746 - val_accuracy: 0.3909 - val_loss: 1.2738 - learning_rate: 2.5000e-04
Epoch 29/50

Epoch 29: val_accuracy improved from 0.39188 to 0.39190, saving model to /content/drive/MyDrive/Introducción a inteligencia artificial/Proyecto/processed/nn_model/best_model.h5



Epoch 29: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
542/542 - 15s - 28ms/step - accuracy: 0.3924 - loss: 1.2743 - val_accuracy: 0.3919 - val_loss: 1.2735 - learning_rate: 2.5000e-04
Epoch 30/50

Epoch 30: val_accuracy did not improve from 0.39190
542/542 - 15s - 27ms/step - accuracy: 0.3928 - loss: 1.2739 - val_accuracy: 0.3913 - val_loss: 1.2730 - learning_rate: 1.2500e-04
Epoch 31/50

Epoch 31: val_accuracy did not improve from 0.39190
542/542 - 20s - 38ms/step - accuracy: 0.3927 - loss: 1.2738 - val_accuracy: 0.3918 - val_loss: 1.2729 - learning_rate: 1.2500e-04
Epoch 32/50

Epoch 32: val_accuracy did not improve from 0.39190
542/542 - 15s - 27ms/step - accuracy: 0.3921 - loss: 1.2737 - val_accuracy: 0.3914 - val_loss: 1.2728 - learning_rate: 1.2500e-04
Epoch 33/50

Epoch 33: val_accuracy did not improve from 0.39190
542/542 - 20s - 38ms/step - accuracy: 0.3923 - loss: 1.2737 - val_accuracy: 0.3915 - val_loss: 1.2728 - learning_rate: 1.2500e-04
Epoch 34/50


542/542 - 15s - 28ms/step - accuracy: 0.3929 - loss: 1.2735 - val_accuracy: 0.3923 - val_loss: 1.2728 - learning_rate: 1.2500e-04
Epoch 35/50

Epoch 35: val_accuracy did not improve from 0.39227
542/542 - 15s - 27ms/step - accuracy: 0.3929 - loss: 1.2736 - val_accuracy: 0.3915 - val_loss: 1.2726 - learning_rate: 1.2500e-04
Epoch 36/50

Epoch 36: val_accuracy improved from 0.39227 to 0.39230, saving model to /content/drive/MyDrive/Introducción a inteligencia artificial/Proyecto/processed/nn_model/best_model.h5


542/542 - 15s - 28ms/step - accuracy: 0.3927 - loss: 1.2735 - val_accuracy: 0.3923 - val_loss: 1.2726 - learning_rate: 1.2500e-04
Epoch 37/50

Epoch 37: val_accuracy did not improve from 0.39230
542/542 - 22s - 41ms/step - accuracy: 0.3935 - loss: 1.2733 - val_accuracy: 0.3921 - val_loss: 1.2726 - learning_rate: 1.2500e-04
Epoch 38/50

Epoch 38: val_accuracy did not improve from 0.39230

Epoch 38: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
542/542 - 16s - 29ms/step - accuracy: 0.3930 - loss: 1.2733 - val_accuracy: 0.3921 - val_loss: 1.2726 - learning_rate: 1.2500e-04
Epoch 39/50

Epoch 39: val_accuracy did not improve from 0.39230
542/542 - 21s - 38ms/step - accuracy: 0.3928 - loss: 1.2733 - val_accuracy: 0.3921 - val_loss: 1.2724 - learning_rate: 6.2500e-05
Epoch 40/50

Epoch 40: val_accuracy did not improve from 0.39230
542/542 - 15s - 27ms/step - accuracy: 0.3934 - loss: 1.2732 - val_accuracy: 0.3921 - val_loss: 1.2724 - learning_rate: 6.2500e-05
Epoch 41/50




Epoch 42: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
542/542 - 15s - 28ms/step - accuracy: 0.3931 - loss: 1.2729 - val_accuracy: 0.3924 - val_loss: 1.2723 - learning_rate: 6.2500e-05
Epoch 43/50

Epoch 43: val_accuracy did not improve from 0.39236
542/542 - 15s - 27ms/step - accuracy: 0.3935 - loss: 1.2729 - val_accuracy: 0.3921 - val_loss: 1.2722 - learning_rate: 3.1250e-05
Epoch 44/50

Epoch 44: val_accuracy did not improve from 0.39236
542/542 - 15s - 27ms/step - accuracy: 0.3933 - loss: 1.2729 - val_accuracy: 0.3922 - val_loss: 1.2722 - learning_rate: 3.1250e-05
Epoch 45/50

Epoch 45: val_accuracy did not improve from 0.39236
542/542 - 15s - 27ms/step - accuracy: 0.3936 - loss: 1.2728 - val_accuracy: 0.3922 - val_loss: 1.2723 - learning_rate: 3.1250e-05
Epoch 46/50

Epoch 46: val_accuracy did not improve from 0.39236

Epoch 46: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
542/542 - 20s - 38ms/step - accuracy: 0.3933 - loss: 1.2728 - v

# **Generación de predicciones y archivo de envío**

En esta sección se utiliza el mejor modelo entrenado para realizar predicciones sobre el conjunto de prueba y generar el archivo de envío final.

---

### **Procedimiento**
- Se cargó el **mejor modelo guardado** (`best_model.h5`) desde la carpeta de resultados.  
- Se obtuvieron las **probabilidades de predicción** sobre el conjunto `X_test` mediante `model.predict()`.  
- Se aplicó `argmax` para convertir las probabilidades en las **clases más probables**.  
- Los índices resultantes se transformaron a sus **etiquetas originales** usando el `LabelEncoder` cargado previamente.  
- Se construyó un **DataFrame de submission** con las columnas `ID` y `RENDIMIENTO_GLOBAL`, siguiendo el formato del archivo de ejemplo.  
- Finalmente, se guardó el resultado como `submission_nn.csv` en la carpeta del proyecto.


In [ ]:
# cargo el mejor modelo guardado (seguro que existe)
from tensorflow.keras.models import load_model
best_model_path = os.path.join(OUT_MODEL, 'best_model.h5')
best = load_model(best_model_path)

# predecir probabilidades y tomar argmax
pred_proba = best.predict(X_test, batch_size=2048, verbose=1)
pred_idx = np.argmax(pred_proba, axis=1)

# mapear índices a etiquetas originales
pred_labels = le.inverse_transform(pred_idx)

# construir DataFrame de submission
submission = pd.DataFrame({
    'ID': sample_sub['ID'],
    'RENDIMIENTO_GLOBAL': pred_labels
})

OUT_SUB = '/content/drive/MyDrive/Introducción a inteligencia artificial/Proyecto/submission_nn.csv'
submission.to_csv(OUT_SUB, index=False)
print("Submission guardado en:", OUT_SUB)
display(submission.head())


145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step
Submission guardado en: /content/drive/MyDrive/Introducción a inteligencia artificial/Proyecto/submission_nn.csv


,ID,RENDIMIENTO_GLOBAL
0,550236,alto
1,98545,medio-alto
2,499179,alto
3,782980,bajo
4,785185,bajo


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"federicooq","key":"47823ee98b1a94e0a4b0e2c89bb6430b"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions list | head

ref                                                                              deadline             category                reward  teamCount  userHasEntered  
-------------------------------------------------------------------------------  -------------------  ---------------  -------------  ---------  --------------  
https://www.kaggle.com/competitions/arc-prize-2025                               2025-11-03 23:59:00  Featured         1,000,000 Usd       1224           False  
https://www.kaggle.com/competitions/jigsaw-agile-community-rules                 2025-10-23 23:59:00  Featured           100,000 Usd       2468           False  
https://www.kaggle.com/competitions/hull-tactical-market-prediction              2025-12-15 23:59:00  Featured           100,000 Usd       1302           False  
https://www.kaggle.com/competitions/google-code-golf-2025                        2025-10-30 23:59:00  Research           100,000 Usd       1050           False  
https://www.kaggle.com/compe

In [ ]:
!cp '/content/drive/MyDrive/Introducción a inteligencia artificial/Proyecto/submission_nn.csv' submission.csv
!kaggle competitions submit -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia -f submission.csv -m "Red neuronal básica - primera versión"


100% 3.93M/3.93M [00:00<00:00, 4.96MB/s]
Successfully submitted to UDEA/ai4eng 20252 - Pruebas Saber Pro Colombia

In [ ]:
# crear directorio y mover el token kaggle
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# verificar conexión con Kaggle
!kaggle competitions list | head


ref                                                                              deadline             category                reward  teamCount  userHasEntered  
-------------------------------------------------------------------------------  -------------------  ---------------  -------------  ---------  --------------  
https://www.kaggle.com/competitions/arc-prize-2025                               2025-11-03 23:59:00  Featured         1,000,000 Usd       1224           False  
https://www.kaggle.com/competitions/jigsaw-agile-community-rules                 2025-10-23 23:59:00  Featured           100,000 Usd       2468           False  
https://www.kaggle.com/competitions/hull-tactical-market-prediction              2025-12-15 23:59:00  Featured           100,000 Usd       1302           False  
https://www.kaggle.com/competitions/google-code-golf-2025                        2025-10-30 23:59:00  Research           100,000 Usd       1050           False  
https://www.kaggle.com/compe